# 95% CI for Human RTs — Human–Human 2P2G & 2P3G

Computes 95% CIs (mean ± 1.96·SEM) for first-move RT and inter-move RT, focusing on human–human trials in 2P2G and 2P3G. Outputs combined (both game types) and by game type. Also provides ±3 SD trimmed versions.

In [23]:
import pandas as pd
import numpy as np
import ast, json
from pathlib import Path

data_path = Path('../dataAnalysis/human-human-with-gpt-fallback/combined_experiment_data.csv')
assert data_path.exists(), f'Missing data file: {data_path}'
df = pd.read_csv(data_path)
df_hh = df[(df['partnerAgentType']=='human') & (df['experimentType'].isin(['2P2G','2P3G']))].copy()
len(df_hh)

622

In [24]:
def parse_array(x):
    if isinstance(x, (list, tuple, np.ndarray)):
        return list(x)
    if pd.isna(x):
        return []
    s = str(x).strip().replace('null','None')
    try:
        arr = ast.literal_eval(s)
    except Exception:
        try: arr = json.loads(str(x))
        except Exception: return []
    return [v for v in arr if v is not None]

def get_focal_rt(row):
    cp = row.get('currentPlayer')
    if pd.isna(cp): return []
    cp = int(cp)
    return parse_array(row['player1RT'] if cp==1 else row['player2RT'])

recs = []
for _, r in df_hh.iterrows():
    rts = get_focal_rt(r)
    for i, rt in enumerate(rts, start=1):
        try: rt = float(rt)
        except: continue
        recs.append({
            'participantId': r['participantId'],
            'roomId': r['roomId'],
            'experimentType': r['experimentType'],
            'trialIndex': r['trialIndex'],
            'move_index': i,
            'rt_ms': rt,
        })
rt_long = pd.DataFrame(recs)
len(rt_long)

8303

In [25]:
# First-move and inter-move tables
first_move_recs = []
for (pid,rid,et,ti), g in rt_long.groupby(['participantId','roomId','experimentType','trialIndex']):
    g = g.sort_values('move_index')
    if len(g)==0: continue
    fm = g.iloc[0]
    first_move_recs.append({
        'participantId': pid, 'roomId': rid, 'experimentType': et, 'trialIndex': ti,
        'first_move_rt_ms': fm['rt_ms']
    })
first_move = pd.DataFrame(first_move_recs)

im_parts = []
for keys, g in rt_long.groupby(['participantId','roomId','experimentType','trialIndex']):
    g = g.sort_values('move_index').copy()
    g['inter_rt_ms'] = g['rt_ms'].diff()
    g['inter_move_index'] = g['move_index']
    gg = g[g['inter_rt_ms'].notna() & (g['inter_rt_ms'] > 0)].copy()
    if len(gg)>0: im_parts.append(gg[['participantId','roomId','experimentType','trialIndex','inter_move_index','inter_rt_ms']])
imrt_long = pd.concat(im_parts, ignore_index=True) if im_parts else pd.DataFrame(columns=['participantId','roomId','experimentType','trialIndex','inter_move_index','inter_rt_ms'])
len(first_move), len(imrt_long)

(622, 7681)

In [26]:
def std_trim_df(df, col, k=3.0):
    if df is None or len(df)==0: return df.copy()
    m, s = df[col].mean(), df[col].std(ddof=1)
    if pd.isna(s) or s==0: return df.copy()
    lo, hi = m-k*s, m+k*s
    return df[(df[col]>=lo) & (df[col]<=hi)].copy()

def ci95(series):
    s = pd.to_numeric(pd.Series(series).dropna(), errors='coerce').dropna()
    n = len(s)
    if n==0: return np.nan, np.nan, np.nan, 0
    m = s.mean(); sd = s.std(ddof=1); sem = sd/np.sqrt(n)
    return m, m-1.96*sem, m+1.96*sem, n

# Combined raw
m1,l1,h1,n1 = ci95(first_move['first_move_rt_ms'])
m2,l2,h2,n2 = ci95(imrt_long['inter_rt_ms'])
combined_raw = pd.DataFrame([
    {'measure':'first_move_rt_ms','mean_ms':m1,'low_95':l1,'high_95':h1,'n':n1},
    {'measure':'inter_move_rt_ms','mean_ms':m2,'low_95':l2,'high_95':h2,'n':n2},
])
# combined_raw.to_csv('dataAnalysis/human_rt_95ci_combined_raw.csv', index=False)
combined_raw

,measure,mean_ms,low_95,high_95,n
0,first_move_rt_ms,1203.258842,1120.015543,1286.502142,622
1,inter_move_rt_ms,346.942586,337.171558,356.713613,7681


In [27]:
# Combined ±3 SD trimmed
fm_std = std_trim_df(first_move, 'first_move_rt_ms', 3.0)
imrt_std = std_trim_df(imrt_long, 'inter_rt_ms', 3.0)
m1t,l1t,h1t,n1t = ci95(fm_std['first_move_rt_ms'])
m2t,l2t,h2t,n2t = ci95(imrt_std['inter_rt_ms'])
combined_std = pd.DataFrame([
    {'measure':'first_move_rt_ms','mean_ms':m1t,'low_95':l1t,'high_95':h1t,'n':n1t},
    {'measure':'inter_move_rt_ms','mean_ms':m2t,'low_95':l2t,'high_95':h2t,'n':n2t},
])
# combined_std.to_csv('dataAnalysis/human_rt_95ci_combined_stdtrim.csv', index=False)
combined_std

,measure,mean_ms,low_95,high_95,n
0,first_move_rt_ms,1092.753289,1037.285201,1148.221378,608
1,inter_move_rt_ms,297.062575,291.546781,302.578368,7511


In [28]:
# By game type (raw)
rows = []
for et, g in first_move.groupby('experimentType'):
    m,l,h,n = ci95(g['first_move_rt_ms']); rows.append({'experimentType':et,'measure':'first_move_rt_ms','mean_ms':m,'low_95':l,'high_95':h,'n':n})
for et, g in imrt_long.groupby('experimentType'):
    m,l,h,n = ci95(g['inter_rt_ms']); rows.append({'experimentType':et,'measure':'inter_move_rt_ms','mean_ms':m,'low_95':l,'high_95':h,'n':n})
by_type_raw = pd.DataFrame(rows)
# by_type_raw.to_csv('dataAnalysis/human_rt_95ci_by_type_raw.csv', index=False)
by_type_raw

,experimentType,measure,mean_ms,low_95,high_95,n
0,2P2G,first_move_rt_ms,1266.566308,1134.359153,1398.773463,279
1,2P3G,first_move_rt_ms,1151.763848,1045.962812,1257.564885,343
2,2P2G,inter_move_rt_ms,335.772755,322.409963,349.135546,3296
3,2P3G,inter_move_rt_ms,355.338426,341.483991,369.192862,4385


In [29]:
# By game type (±3 SD)
rows = []
for et, g in fm_std.groupby('experimentType'):
    m,l,h,n = ci95(g['first_move_rt_ms']); rows.append({'experimentType':et,'measure':'first_move_rt_ms','mean_ms':m,'low_95':l,'high_95':h,'n':n})
for et, g in imrt_std.groupby('experimentType'):
    m,l,h,n = ci95(g['inter_rt_ms']); rows.append({'experimentType':et,'measure':'inter_move_rt_ms','mean_ms':m,'low_95':l,'high_95':h,'n':n})
by_type_std = pd.DataFrame(rows)
# by_type_std.to_csv('dataAnalysis/human_rt_95ci_by_type_stdtrim.csv', index=False)
by_type_std

,experimentType,measure,mean_ms,low_95,high_95,n
0,2P2G,first_move_rt_ms,1167.481752,1077.121864,1257.841640,274
1,2P3G,first_move_rt_ms,1031.449102,963.466586,1099.431617,334
2,2P2G,inter_move_rt_ms,298.732016,290.169753,307.294280,3239
3,2P3G,inter_move_rt_ms,295.796816,288.591741,303.001892,4272


## Participant-Averaged 95% CIs
Compute per-participant means first, then 95% CIs across participants (combined and by type), for raw and ±3 SD trimmed data.

In [30]:
# Participant-level means (raw)
fm_part = (first_move.groupby('participantId')['first_move_rt_ms'].mean().reset_index())
im_part = (imrt_long.groupby('participantId')['inter_rt_ms'].mean().reset_index())

m1,l1,h1,n1 = ci95(fm_part['first_move_rt_ms'])
m2,l2,h2,n2 = ci95(im_part['inter_rt_ms'])
combined_part_raw = pd.DataFrame([
    {'measure':'first_move_rt_ms','mean_ms':m1,'low_95':l1,'high_95':h1,'n_participants':n1},
    {'measure':'inter_move_rt_ms','mean_ms':m2,'low_95':l2,'high_95':h2,'n_participants':n2},
])
# combined_part_raw.to_csv('dataAnalysis/human_rt_95ci_combined_participant_raw.csv', index=False)
combined_part_raw

,measure,mean_ms,low_95,high_95,n_participants
0,first_move_rt_ms,1234.806456,968.893997,1500.718916,39
1,inter_move_rt_ms,350.454159,314.541352,386.366966,39


In [32]:
# Participant-level means by game type (raw)
rows = []
for et, fmg in first_move.groupby('experimentType'):
    fm_part_t = fmg.groupby('participantId')['first_move_rt_ms'].mean()
    m,l,h,n = ci95(fm_part_t)
    rows.append({'experimentType':et,'measure':'first_move_rt_ms','mean_ms':m,'low_95':l,'high_95':h,'n_participants':n})
for et, img in imrt_long.groupby('experimentType'):
    im_part_t = img.groupby('participantId')['inter_rt_ms'].mean()
    m,l,h,n = ci95(im_part_t)
    rows.append({'experimentType':et,'measure':'inter_move_rt_ms','mean_ms':m,'low_95':l,'high_95':h,'n_participants':n})
by_type_part_raw = pd.DataFrame(rows)
# by_type_part_raw.to_csv('dataAnalysis/human_rt_95ci_by_type_participant_raw.csv', index=False)
by_type_part_raw

,experimentType,measure,mean_ms,low_95,high_95,n_participants
0,2P2G,first_move_rt_ms,1281.351007,1005.278654,1557.423361,39
1,2P3G,first_move_rt_ms,1145.860640,888.051976,1403.669304,33
2,2P2G,inter_move_rt_ms,341.806345,304.018561,379.594130,39
3,2P3G,inter_move_rt_ms,358.384872,318.242533,398.527210,33


In [33]:
# Participant-level means (±3 SD trimmed)
fm_std = std_trim_df(first_move, 'first_move_rt_ms', 3.0)
imrt_std = std_trim_df(imrt_long, 'inter_rt_ms', 3.0)
fm_part_std = fm_std.groupby('participantId')['first_move_rt_ms'].mean().reset_index()
im_part_std = imrt_std.groupby('participantId')['inter_rt_ms'].mean().reset_index()

m1t,l1t,h1t,n1t = ci95(fm_part_std['first_move_rt_ms'])
m2t,l2t,h2t,n2t = ci95(im_part_std['inter_rt_ms'])
combined_part_std = pd.DataFrame([
    {'measure':'first_move_rt_ms','mean_ms':m1t,'low_95':l1t,'high_95':h1t,'n_participants':n1t},
    {'measure':'inter_move_rt_ms','mean_ms':m2t,'low_95':l2t,'high_95':h2t,'n_participants':n2t},
])
# combined_part_std.to_csv('dataAnalysis/human_rt_95ci_combined_participant_stdtrim.csv', index=False)
combined_part_std

,measure,mean_ms,low_95,high_95,n_participants
0,first_move_rt_ms,1154.804900,948.654485,1360.955316,39
1,inter_move_rt_ms,302.679571,277.176099,328.183042,39


In [34]:
# Participant-level means by game type (±3 SD)
rows = []
for et, fmg in fm_std.groupby('experimentType'):
    fm_part_t = fmg.groupby('participantId')['first_move_rt_ms'].mean()
    m,l,h,n = ci95(fm_part_t)
    rows.append({'experimentType':et,'measure':'first_move_rt_ms','mean_ms':m,'low_95':l,'high_95':h,'n_participants':n})
for et, img in imrt_std.groupby('experimentType'):
    im_part_t = img.groupby('participantId')['inter_rt_ms'].mean()
    m,l,h,n = ci95(im_part_t)
    rows.append({'experimentType':et,'measure':'inter_move_rt_ms','mean_ms':m,'low_95':l,'high_95':h,'n_participants':n})
by_type_part_std = pd.DataFrame(rows)
# by_type_part_std.to_csv('dataAnalysis/human_rt_95ci_by_type_participant_stdtrim.csv', index=False)
by_type_part_std

,experimentType,measure,mean_ms,low_95,high_95,n_participants
0,2P2G,first_move_rt_ms,1208.627106,994.207675,1423.046537,39
1,2P3G,first_move_rt_ms,1082.727720,872.310804,1293.144636,33
2,2P2G,inter_move_rt_ms,305.613247,276.187547,335.038947,39
3,2P3G,inter_move_rt_ms,300.472576,273.406953,327.538198,33
